In [2]:
from box import ConfigBox

In [1]:
print("ehll")

ehll


In [3]:
from src.constants import CONFIG_PATH

ModuleNotFoundError: No module named 'src'

In [1]:
cd "Documents/Projects/ECommerce Customer Churn Prediction"

[WinError 3] The system cannot find the path specified: 'Documents/Projects/ECommerce Customer Churn Prediction'
c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\notebooks


c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction\\notebooks'

In [5]:
cd ..

c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction


c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction\\notebooks'

In [2]:
import os 
os.chdir("..")

In [3]:
pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction'

In [4]:
from src.constants import CONFIG_PATH,PARAMS_PATH,SCHEMA_PATH
from src.utils.common import read_yaml
from src.entity.data_ingestion import DataIngestionConfig

print(DataIngestionConfig.__annotations__)


class ConfigurationManager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config=read_yaml(config_path),
        self.params=read_yaml(params_path),
        self.schema=read_yaml(schema_path)
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config[0].data
        data_ingestion=DataIngestionConfig(
            kaggle_dataset=config.kaggle_source_path,
            gdrive_dataset=config.gdrive_source_path,
            local_dataset=config.raw,
            staging_dataset=config.interim
        )
        return data_ingestion

{'kaggle_dataset': <class 'str'>, 'gdrive_dataset': <class 'str'>, 'local_dataset': <class 'str'>, 'staging_dataset': <class 'str'>}


In [5]:
config=ConfigurationManager()
print(config.get_data_ingestion_config())

[2025-02-21 18:01:25,844 : INFO: common: yaml file config\config.yaml loaded successfully!]
[2025-02-21 18:01:25,854 : INFO: common: yaml file config\params.yaml loaded successfully!]
[2025-02-21 18:01:25,859 : INFO: common: yaml file config\schema.yaml loaded successfully!]
DataIngestionConfig(kaggle_dataset='ankitverma2010/ecommerce-customer-churn-analysis-and-prediction', gdrive_dataset='https://drive.google.com/uc?id=1L33uQvZl0D2gNnu79tbJnjuX0ZAzr9_C', local_dataset='data/raw/', staging_dataset='data/interim/')


In [7]:
import os
import pandas as pd
import kagglehub
import shutil
import gdown

class DataIngestion:
    def __init__(self):
        self.config=ConfigurationManager().get_data_ingestion_config()
    def download_kaggle_data(self):
        path = kagglehub.dataset_download(self.config.kaggle_dataset,force_download=True)
        if(os.path.exists(self.config.local_dataset+"1/") and os.path.isdir(self.config.local_dataset+"1/")):
            shutil.rmtree(self.config.local_dataset+"1/")
        shutil.move(path,self.config.local_dataset)
        return pd.read_excel(self.config.local_dataset+'1/E Commerce Dataset.xlsx',sheet_name=1)
    def gdrive_data(self):
        #dowload data from public dataset hosted in gdrive
        if(os.path.exists(self.config.local_dataset+"EComm_gdrive_raw_data.xlsx")):
            os.remove(self.config.local_dataset+"EComm_gdrive_raw_data.xlsx")
        gdown.download(self.config.gdrive_dataset,"data/raw/EComm_gdrive_raw_data.xlsx",quiet=False)
        return pd.read_excel(self.config.local_dataset+'EComm_gdrive_raw_data.xlsx',sheet_name=1,engine='openpyxl')
    def run(self):
        kaggle_data_df=self.download_kaggle_data()
        gdrive_data_df=self.gdrive_data()
        
        #Ensuring both datasets have same schema
        total_rows=len(kaggle_data_df)
        half_size=total_rows//2
        
        half_kaggle=kaggle_data_df.iloc[:half_size]
        half_gdrive=gdrive_data_df.iloc[-half_size:]
        
        combined_data=pd.concat([half_kaggle,half_gdrive],ignore_index=True)
        return combined_data.to_csv(self.config.staging_dataset+"Combined_dataset_churn_prediction.csv")
    
        
        
        
            
        
        
        


c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
ingestion=DataIngestion()
df=ingestion.run()
df

[2025-02-21 18:02:36,750 : INFO: common: yaml file config\config.yaml loaded successfully!]
[2025-02-21 18:02:36,757 : INFO: common: yaml file config\params.yaml loaded successfully!]
[2025-02-21 18:02:36,763 : INFO: common: yaml file config\schema.yaml loaded successfully!]


100%|██████████| 532k/532k [00:01<00:00, 433kB/s]

Extracting files...



Downloading...
From: https://drive.google.com/uc?id=1L33uQvZl0D2gNnu79tbJnjuX0ZAzr9_C
To: c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\data\raw\EComm_gdrive_raw_data.xlsx
100%|██████████| 556k/556k [00:00<00:00, 2.07MB/s]


In [ ]:
import shutil
shutil.move(r"C:\Users\asus\.cache\kagglehub\datasets\ankitverma2010\ecommerce-customer-churn-analysis-and-prediction\versions\1")